In [ ]:
!pip install folium pandas


In [ ]:
# from google.colab import files
#uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

csv_yolu = "/content/drive/MyDrive/ybm/bildiriler/jeosit_detaylari.csv"

# CSV'yi oku
df = pd.read_csv(csv_yolu)

# İlk 5 satırı göster
df.head()



,Latitude,Longitude,Jeosit Adı,Kategori / Grup,Öneren(ler),Açıklama
0,40.112519,33.918800,"Cingeyli Kızılı, Ambar Kızılı",VIII,Mustafa Dönmez,Üst Eosen - Oligosen yaşlı karasal çakıltaşı-k...
1,41.472715,34.641166,Boyabat bazalt sütunları,IV,Ediz Kırman,Paleosen yaşlı
2,37.829042,35.911059,Infra- Kambriyen yaşlı Emirgazi Formasyonu tip...,I,"Hüseyin Kozlu, M.Cemal Göncüoğlu",Anadolu’da bulunan ve metamorfik olmayan en es...
3,41.292083,33.102969,Karadere deresi,I,Esen Arpat,Graptolitli düzeyler Arenig katından Blanvirn ...
4,38.465030,36.487161,Doğu Toroslar - Sarız – Halevik Dere,I,"Hüseyin Kozlu, Jean-François Ghienne",1-Buzul çökeliminin tüm özelliklerinin açık ve...


In [ ]:
# Gerekli kütüphaneler
import pandas as pd
import folium
from google.colab import drive

# Google Drive'ı bağla
drive.mount('/content/drive')

# CSV dosyasının yolu
csv_yolu = "/content/drive/MyDrive/ybm/bildiriler/jeosit_detaylari.csv"

# CSV'den veriyi oku
df = pd.read_csv(csv_yolu)

# Harita oluştur: Türkiye merkezli
m = folium.Map(location=[39.0, 35.0], zoom_start=6)

# Jeositleri haritaya ekle
for _, row in df.iterrows():
    try:
        lat = float(row['Latitude'])
        lon = float(row['Longitude'])
        tooltip = row['Jeosit Adı']
        popup = folium.Popup(f"""
            <b>{row['Jeosit Adı']}</b><br>
            <b>Kategori:</b> {row['Kategori / Grup']}<br>
            <b>Öneren:</b> {row['Öneren(ler)']}<br>
            <b>Açıklama:</b> {row['Açıklama']}
        """, max_width=300)

        folium.Marker(
            location=[lat, lon],
            tooltip=tooltip,
            popup=popup,
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)
    except:
        continue  # Hatalı satır varsa atla

# Haritayı HTML olarak kaydet
m.save('/content/drive/MyDrive/ybm/bildiriler/jeosit_harita.html')

# Haritayı göster
m


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import folium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

# CSV'den veri oku
df = pd.read_csv('jeosit_detaylari.csv')

# Folium haritası oluştur
m = folium.Map(location=[39.0, 35.0], zoom_start=6)

for _, row in df.iterrows():
    tooltip = f"{row['Jeosit Adı']}"
    popup = folium.Popup(f"""
    <b>{row['Jeosit Adı']}</b><br>
    <b>Kategori:</b> {row['Kategori / Grup']}<br>
    <b>Öneren:</b> {row['Öneren(ler)']}<br>
    <b>Açıklama:</b> {row['Açıklama']}
    """, max_width=300)

    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        tooltip=tooltip,
        popup=popup,
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Haritayı geçici olarak kaydet
map_file = "jeosit_harita.html"
m.save(map_file)

# Chrome başlatma ayarları
options = Options()
options.headless = True
options.add_argument("--window-size=1200x800")

# WebDriver ile HTML dosyasını aç ve ekran görüntüsü al
driver = webdriver.Chrome(options=options)
driver.get(f"file://{map_file}")

# Yeterince yüklenmesi için bekle
time.sleep(3)

# Ekran görüntüsü al
driver.save_screenshot("jeosit_harita.png")
driver.quit()


In [ ]:
# Kategori etiketlerini sadece ad ve sayı olarak hazırlayalım (açıklama olmadan)
category_counts = df['Kategori / Grup'].value_counts().sort_index()
category_labels = [
    f"{cat} ({category_counts[cat]})"
    for cat in sorted(category_counts.index)
]

# Yeniden sıralı kategoriler
df_sorted["Kategori / Grup"] = pd.Categorical(
    df_sorted["Kategori / Grup"],
    categories=sorted(category_counts.index),
    ordered=True
)

# Grafik çizimi
plt.figure(figsize=(12, 8))
scatter_plot = sns.scatterplot(
    data=df_sorted,
    x="Longitude",
    y="Latitude",
    hue="Kategori / Grup",
    palette="tab10",
    s=120,
    edgecolor="k",
)

# Legend ayarları (sadece ad ve sayı)
handles, _ = scatter_plot.get_legend_handles_labels()
plt.legend(
    handles=handles[1:],
    labels=category_labels,
    title="Kategori / Grup (Adet)",
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    borderaxespad=0.
)

# Başlık ve eksen etiketleri
plt.title("Jeosit Alanlarının Kategoriye Göre Dağılımı (Ad + Sayı)", fontsize=16)
plt.xlabel("Boylam (Longitude)")
plt.ylabel("Enlem (Latitude)")
plt.tight_layout()
plt.show()


In [ ]:
import folium
from folium import Map, CircleMarker
from folium.plugins import MarkerCluster

# Temel harita ayarları
map_center = [df["Latitude"].mean(), df["Longitude"].mean()]
folium_map = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")

# Kategorilere göre renk haritası
import matplotlib.cm as cm
import matplotlib.colors as colors

categories = sorted(df["Kategori / Grup"].unique())
colormap = cm.get_cmap("tab10", len(categories))
category_colors = {cat: colors.rgb2hex(colormap(i)) for i, cat in enumerate(categories)}

# Marker cluster
marker_cluster = MarkerCluster().add_to(folium_map)

# Kategori bazlı noktaları ekleyelim
for _, row in df.iterrows():
    category = row["Kategori / Grup"]
    color = category_colors.get(category, "#000000")
    popup_text = f"{row['Jeosit Adı']}"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        popup=popup_text,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.8
    ).add_to(marker_cluster)

# Renkli legend üretelim
legend_html = """
<div style="position: fixed;
     bottom: 30px; left: 30px; width: 250px; height: auto;
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
<b>Kategoriler</b><br>
"""
for cat in categories:
    color = category_colors[cat]
    legend_html += f'<i style="background:{color};width:12px;height:12px;float:left;margin-right:6px;"></i>{cat}<br>'
legend_html += "</div>"

# Legend'i haritaya ekle
folium_map.get_root().html.add_child(folium.Element(legend_html))

# HTML'e kaydetmeden sadece haritayı döndür
folium_map


In [ ]:
import random

# Kategoriler için daha farklı renkler üretelim (örneğin 10 adet için)
def generate_distinct_colors(n):
    colors = []
    for _ in range(n):
        colors.append("#{:06x}".format(random.randint(0x444444, 0xFFFFFF)))
    return colors

# Yeni renk paleti
distinct_colors = generate_distinct_colors(len(categories))
category_colors_custom = dict(zip(categories, distinct_colors))

# Yeni harita
folium_map_custom = folium.Map(location=map_center, zoom_start=6, tiles="CartoDB positron")
marker_cluster = MarkerCluster().add_to(folium_map_custom)

# Noktaları yeni renklerle haritaya ekleyelim
for _, row in df.iterrows():
    category = row["Kategori / Grup"]
    color = category_colors_custom.get(category, "#000000")
    popup_text = f"{row['Jeosit Adı']}"
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        popup=popup_text,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.85
    ).add_to(marker_cluster)

# Yeni legend
legend_html_custom = """
<div style="position: fixed;
     bottom: 30px; left: 30px; width: 260px; height: auto;
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
<b>Kategoriler (Yeni Renklerle)</b><br>
"""
for cat in categories:
    color = category_colors_custom[cat]
    legend_html_custom += f'<i style="background:{color};width:12px;height:12px;float:left;margin-right:6px;"></i>{cat}<br>'
legend_html_custom += "</div>"

# Legend'i haritaya ekle
folium_map_custom.get_root().html.add_child(folium.Element(legend_html_custom))

# Haritayı göster
folium_map_custom
